In [1]:
import datetime

import numpy as np
import pandas as pd
import pycountry

In [2]:
ds = pd.read_excel('cleanedLocalISO.xlsx', sheet_name='Reviews', decimal=",", engine='openpyxl')
ds.describe(include='all', datetime_is_numeric=True).T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
localID,84022,99,MAG001,7883,NaN,NaN,NaN,NaN,NaN,NaN,NaN
localISO,84022,24,UK,23071,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cluster,84022.0,NaN,NaN,NaN,13.609317,0.0,3.0,12.0,24.0,39.0,11.740607
globalRating,84022.0,NaN,NaN,NaN,4.485135,4.0,4.5,4.5,4.5,5.0,0.178728
positionOnRanking,84022.0,NaN,NaN,NaN,3.905096,1.0,1.0,2.0,6.0,55.0,4.859406
sitesOnRanking,84022.0,NaN,NaN,NaN,743.663219,5.0,154.0,487.0,1186.0,3837.0,793.367315
totalReviews,84022.0,NaN,NaN,NaN,41060.391124,5179.0,14152.0,24454.0,51324.0,163828.0,43532.779269
userName,84022,64982,Malgorzata@Margo7850p,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userLocation,71718,12460,"London, UK",3391,NaN,NaN,NaN,NaN,NaN,NaN,NaN
userContributions,84022.0,NaN,NaN,NaN,479.916808,0.0,20.0,66.0,216.0,607732.0,7281.411885


In [3]:
dsLocation = ds[['userName','userLocation']] # subset only to the features that I need
dsLocation.drop_duplicates(inplace=True) # Removing duplicates
dsLocation.dropna(inplace=True) # Removing users with null as a userLocation

/tmp/ipykernel_7307/313522914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsLocation.drop_duplicates(inplace=True) # Removing duplicates
/tmp/ipykernel_7307/313522914.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsLocation.dropna(inplace=True) # Removing users with null as a userLocation


In [4]:
# Splitting the userLocation in order to perfom an analysis only on the last value after the comma which usually ISO or name of a country
dsCopy = dsLocation.copy()
dsCopy['userLocation'] = ds['userLocation'].str.split(',')
dsCopy.dropna(subset=['userLocation'], inplace=True)
list = dsCopy['userLocation'].tolist()
print(len(list))

54454


In [5]:

def do_fuzzy_search(country: str) -> str:
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return country
    else:
        return result[0].alpha_2

Need to add a filter on IN because is not India but indianapolis, one on IL because is not israel but illinois, ca is california not canada, pa is pennsylvania not panama, ma is massachusetts not marocco, ga is georiga not gambia, nc is north carolina

In [6]:
copyList = list.copy() # deep copy in order to not change list when we write on copylist

for i in range(len(list)):
    # copy the value of the contry/ISO
    country = list[i][-1].strip() 
    # check if is ISO or not
    if len(country) > 2:
        copyList[i] = do_fuzzy_search(country)
    # if is lenght is 2 check if is an ISO code or not
    elif len(country) == 2:
        country = country.upper()
        if country == 'UK': # uk is gb but since on the rest of the dataset is called uk we keep uk
            copyList[i] = country
        else:    
            copyList[i] = 'US' # if we have another value we keep that value es: new york, NY we drop ny and keep new york to pass to geopy next step 
    elif len(country) == 1:
        copyList[i] = np.nan
    print(copyList[i])

US
IE
ES
ES
US
UK
UA
FR
US
CH
GR
ES
NL
PL
US
US
CA
NL
US
UK
ES
US
US
UK
DE
DE
GR
IT
US
UK
CR
ES
UK
NO
US
US
UK
IE
ES
ZA
ES
UK
US
UK
US
HR
US
US
FR
GB
EE
ES
GR
MX
US
US
IN
US
ES
ES
ES
UK
ES
UK
UK
US
FR
US
US
CA
UK
AU
IT
UK
CA
UK
UK
UK
IT
ES
GB
NL
NL
UK
ES
UK
CA
NL
UK
LU
EG
UK
MY
ES
US
UK
AU
SI
UK
TH
UK
UK
BE
CN
LU
CN
ES
UK
ID
PL
ES
US
US
US
US
US
US
AU
US
UK
CA
US
AU
US
US
UK
UK
US
US
IE
UK
UK
US
UK
US
UK
NZ
UK
IN
US
ES
US
US
UK
US
AU
UK
SG
US
UK
EG
BE
RU
US
CA
SE
US
ES
US
GB
UK
GR
US
US
UK
CY
UK
IN
UK
US
UK
US
CA
US
TR
IR
UK
GR
UK
ES
UK
UK
IN
US
GR
UK
US
US
US
US
US
US
GR
GB
US
GB
UK
CA
CA
BG
DE
US
IE
IN
US
UK
UK
UK
SG
US
UK
UK
UK
UK
UK
US
NL
US
US
BE
US
UK
UK
UK
UK
BE
CA
US
UK
US
TH
BE
IT
US
UK
UK
UK
AU
UK
US
GB
MX
US
MX
UK
US
DE
UK
UK
UK
UK
US
Boston
UK
US
UK
UK
US
MX
AU
US
UK
DE
US
GE
US
US
ZA
UK
UK
FR
UK
UK
UK
UK
US
GB
BG
US
BE
CA
UK
UK
UK
GB
UK
US
ES
US
UK
UK
UK
MX
UK
US
NL
US
UK
UK
US
US
KW
UK
US
SG
US
US
AR
US
UK
UK
Potters bar
RO
US
US
IE
UK
US
UK
UK
US
US
US
UK
US
US
UK
CA
AT


In [7]:
print(len(copyList) == len(list))

True


In [8]:
dsCopy['userISO'] = copyList
dsCopy.head()

,userName,userLocation,userISO
0,Maurice M@MauriceM171,"[Indianapolis, IN]",US
1,nickchamb@nickchamb,"[Bandon, Ireland]",IE
2,Arthur Villon@arthur4588,"[Valencia, Spain]",ES
4,Kamal K Sirra@Sirra63,"[Madrid, Spain]",ES
5,paul n@pauln3630,"[Towaco, NJ]",US


In [9]:
dsCopy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54454 entries, 0 to 84021
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userName      54454 non-null  object
 1   userLocation  54454 non-null  object
 2   userISO       54448 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [10]:
# I do this here just to don't damage too much performance on the for loop
dsCopy['userISO'] = np.where(dsCopy['userISO'] == 'GB', 'UK', dsCopy['userISO'])

In [11]:
dsCopy.to_excel('locationFirst.xlsx', index=False, sheet_name='Reviews')

print(datetime.datetime.now())

2022-12-21 11:39:35.360038
